In [111]:
from pathlib import Path
import cv2 
from ultralytics import YOLO
import torch

In [112]:
print(f"Is CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    

Is CUDA available: True
GPU: NVIDIA GeForce GTX 1650


In [113]:
# yolo11n.pt  - Nano
# yolo11s.pt  - Small
# yolo11m.pt  - Medium 
# yolo11l.pt  - Large 
# yolo11x.pt  - Extra Large

model = YOLO("yolo11x")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = YOLO("yolo11x.pt")
model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(384, eps=0.001, momentum=0.03, affine=True, track_

In [114]:
video_path = Path("/home/marco/Documents/smart-cooler-monitor/videos/Dataset-videos/cheetos_crunchy_56.7g_v1/clip_cam4_cheetos_crunchy_56.7g_v1_20250829_095416.mp4")
cap = cv2.VideoCapture(video_path)


In [115]:
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Resolution: {width}x{height}")

original_fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Original fps: {original_fps}")


Resolution: 640x480
Original fps: 30.0


In [116]:
if not cap.isOpened():
    print("Could not open the video")
else:

    paused = False
    frame_count = 0
    
    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                # Restart video
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            
            frame_count += 1
            
            # Detection with YOLO
            results = model(frame, verbose=False)  
            
            # Draw detection
            annotated_frame = results[0].plot()
            
            detections = results[0].boxes
            if detections is not None and len(detections) > 0:
                detection_info = f"Frame {frame_count}: {len(detections)} objects found"
                cv2.putText(annotated_frame, detection_info, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                for i, box in enumerate(detections):
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    class_name = model.names[class_id]
                    if confidence > 0.5:  
                        print(f"  {class_name}: {confidence:.2f}")
            
            # Mostrar el frame con las detecciones
            cv2.imshow('Smart Cooler Tests - YOLO Detection', annotated_frame)
        
        # Control de teclado
        key = cv2.waitKey(33) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('p'):
            paused = not paused
            print("Pausado" if paused else "Reanudado")

    cap.release()
    cv2.destroyAllWindows()
    print("Video cerrado correctamente")

  person: 0.76
  person: 0.62
  person: 0.61
  person: 0.85
  person: 0.78
  person: 0.74
  person: 0.90
  person: 0.88
  person: 0.86
  person: 0.90
  person: 0.55
  person: 0.91
  person: 0.52
  person: 0.93
  person: 0.66
  person: 0.93
  person: 0.92
  person: 0.93
  person: 0.92
  person: 0.94
  person: 0.92
  person: 0.95
  person: 0.92
  person: 0.93
  person: 0.94
  person: 0.94
  person: 0.89
  person: 0.94
  person: 0.94
  person: 0.94
  person: 0.94
  person: 0.93
  person: 0.92
  person: 0.90
  person: 0.57
  person: 0.84
  person: 0.84
  person: 0.81
  person: 0.87
  person: 0.54
  person: 0.88
  person: 0.57
  person: 0.78
  person: 0.73
  person: 0.81
  person: 0.79
Video cerrado correctamente
